# Summary

## Update 20240215
`akshare` stops working for retrieving fundamentals of us stocks. After research, I used https://www.dolthub.com/repositories/post-no-preference/earnings/query/master to get the Book Value, in order to compute PB. The code is modified accordingly. Alternatives APIs are in the Backup section  
Changes include
* Since APIs or query all limit calls of data, I need to first limit the number of stocks in analaysis, instead of retrieving info of all SP500 stocks

## Methodology

**Caveat**
- In my first test (Stock_MA.ipynb), I use the first trading day of MONTH_ACT and first trading day of MONTH_PREV to define the the two ends of the 6m momentum. Here I use the Nth trading day instead. Therefore the result may vary
- If stock splits, the price momentum is unreasonable, I will check it and likely skip that stockthat year
- If using finviz
  - The BV is updated with Q4 results for some stocks
  - I can only use the whole SP500 instead of top 300
  - The momentum filter is not by rank, but by value like 10% increase 
- The SP500 components change by about 20 each year. To accurately apply the magic formula, I should use the sp500 components of that year for backtest, but I don't have it. Here I'm using the latest SP500 list instead. There's an information leak, since those in today's SP500 but not 5 years ago must have been performing well to get into the list in the past 5 years. So I'm picking up those stocks that I know increases a lot in value, and putting in my portfolio to test the performance. 
 - That said, I'm using top300 of sp500 which hopefully are more stable over time. And I have those strict criteria about momentum and PB. So the results should not be too far off


**Methodology (for backtest)** 
Here I get daily price history data and use the price of the action day (The Nth trading day of the action month) to compute price momentum. Also I use a latest full list of SP500, but take only the first N_TOP_BY_MKT_CAP stocks.

The follows are for the backtest. Prediction is similar except I only use one year's data and optimized parameters

Method: 
- Select the N_TOP_BY_MKT_CAP top stocks from SP500
- Starting from 2010, take actions on the Nth trading day of Feb (MONTH_ACT) of each year
- First choose the top 20% (TOP_BY_MMT = 0.2) of stocks ranked by 6 month price momentum (price of the Nth trading day of MONTH_ACT minus price of the Nth trading day of previous MONTH_PREV)
- Then choose the top N_STOCKS ranked by PB. PB = price of the Nth trading day of MONTH_ACT / book value of 2 quarters ago (previous Q3 if MONTH_ACT = Feb)
- On the Nth trading day of MONTH_ACT of each year, sell all stocks from the previous year with tax rate of 10% (TAX_FACTOR = (1 - 0.1)), and buy new stocks with the money by repeating the previous two steps  

**Parameters**
- Tried different N_TOP_BY_MKT_CAP, and 300 is better than the fullist
- Choose the top 40 gives good return among (30, 40, 50)
- Each year, the gain (after 10% lt tax) is better than SPY

In [1]:
from urllib.request import urlopen
import requests
import certifi
import json
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import akshare as ak  # https://github.com/akfamily/akshare

from stocklib import pick_stocks, get_quarter

pd.set_option('display.max_rows', 500)
%matplotlib inline

# Parameters

In [55]:
# Which stock was split in the past year; check back after finding those anomalous momentum
STOCK_SPLIT = ['IDONTKNOW']

## When to buy and sell
MONTH_ACT = 2  
YEAR_ACT = 2024
QUARTER_ACT = f'Q{MONTH_ACT // 3 + 1}'  # Dummy variable. If acting on Feb, the quarter is Q1

# Starting year and month to compute 6m momentum
if MONTH_ACT - 6 > 0:
    MONTH_PREV = MONTH_ACT - 6
    YEAR_PREV = YEAR_ACT 
else:
   YEAR_PREV = YEAR_ACT - 1
   MONTH_PREV = MONTH_ACT + 6  
# The quarter for the BV. Since the ER date differs, this should be 2 quarters ahead of QUARTER_ACT
# so that the BV is available for every stock. If acting in Q1, use previous Q3 
QUARTER_BV = f'Q{MONTH_ACT // 3 + 3}'  
# Trade on which trading day of the month. 
# 1. The maximum of DOM varies by month; so be careful using a big value (> 20)
# 2. Previously I use the first day of the MONTH_ACT for convienience. Now using DOM_TRADING allows me to
# compute on any day of the month. Note that the starting and end dates of 6m momentum is defined as 
# the Nth trading day of MONTH_PREV and that of MONTH_ACT, where N = DOM_TRADING. Therefore they do not
# always fall on the same calendar day of month
# 3. Some stocks don't have price of the previous day in early AM, wait a bit 
DOM_TRADING = 11  
DATE_CUTOFF = f"{YEAR_ACT}-{f'{MONTH_ACT}'.zfill(2)}-01"  # To filter data without recent data

## Filters to pick stocks and compute gains
N_TOP_BY_MKT_CAP = 300  # Choose from the top N of sp500
TOP_BY_MMT = 0.2  # The top fraction of stocks ranked by MMT
MMT_VAR = 'stock_price_mmt_6m'
TAX_FACTOR = (1 - 0.1)
N_STOCKS = 40  # Number of stocks to buy
TOTAL_CASH = 200000.0  # Total cash

## Parameters for both backtest and prediction
# Remove stocks with more than certain quarters with negatives BV. This is to reduce risks based on history.
# When apply the magic formula, only stocks with PB > 0 of the MONTH_ACT are considered
MAX_QUARTERS_NEG_PB = 40

## Backtest only parameters
YEAR_START = 2011  # backtest starting year
CASH_FOR_EACH_STOCK = 1000  # backtest cash for each stock
# Remove stocks with fewer than some quarters
MIN_YEARS_TEST = 3  # Keep stocks with >= MIN_YEARS_TEST years of data

In [3]:
file_name = {
    'fundamental': f'sp500_history_raw_{str(YEAR_ACT)}.csv',
    'price': f'sp500_history_price_raw_{str(YEAR_ACT)}.csv',
}

# Download SP500 price data
Downloading price takes about 0.5hr

In [5]:
## Get fundamental data (seasonal) or price data (daily) 
# Download all SP500 instead of N_TOP_BY_MKT_CAP, since later filters will remove some

is_download = True
is_from_scratch = True  # If starting from scratch and no stock data has been downloaded already
to_download = 'price'  # 'fundamental' or 'price', download fundamental or price data

if is_download:    
    min_row = {
        'fundamental': 3,
        'price': 50,
    }
    anom = {'Failed_PB': [], 'Short': [], 'Failed_P': []}

    # Get stock list (ordered by capital)
    df_sp500_list = pd.read_excel('sp500_fulllist_ranked.xlsx', engine='openpyxl', sheet_name=str(YEAR_ACT))
    stocks = df_sp500_list.loc[df_sp500_list['stock'] != 'GOOG', 'stock'].values.tolist()
    stocks.append('OHI')  # I like OHI

    if not is_from_scratch: 
        print('Read downloaded stocks from local file')
        df_stock_all = pd.read_csv(file_name[to_download])

    # If df_stock_all does not exist, declare it
    try:
        df_stock_all       
    except NameError: 
        df_stock_all = pd.DataFrame()
    stock_downloaded = [] if df_stock_all.empty else df_stock_all.stock.unique()
    print(f'Downloaded {len(stock_downloaded)}, {stock_downloaded}')

    count = 0
    for stock_symbol in stocks:
        if stock_symbol in stock_downloaded:
            continue
        try:
            if to_download == 'price':  # Download price
                df_stock = ak.stock_us_daily(symbol=stock_symbol)
                df_stock = df_stock.reset_index()  # Set the date index to a column
            else:
                print('Wrong variable name')
        except IndexError:
            print(f'Failed for {stock_symbol}')
            anom['Failed_P'].append(stock_symbol)
            continue        
        df_stock['stock'] = stock_symbol
        df_stock_all = pd.concat([df_stock_all, df_stock], ignore_index=True)
        print(f"""{(stock_symbol, 
                    df_stock.date.dt.date.min().isoformat(), 
                    df_stock.date.dt.date.max().isoformat(), 
                    len(df_stock))}""")
        if len(df_stock) < min_row[to_download]:
            anom['Short'].append(stock_symbol)

        count += 1
        if count % 50 == 1:
            print('Saving...')
            df_stock_all.to_csv(file_name[to_download], index=False)    
        time.sleep(1)        

    df_stock_all.to_csv(file_name[to_download], index=False)    

Downloaded 0, []
('MSFT', '1986-03-13', '2024-02-15', 9004)
Saving...
('AAPL', '1984-09-07', '2024-02-15', 9386)
('NVDA', '1999-01-22', '2024-02-15', 5118)
('AMZN', '1997-05-16', '2024-02-15', 6170)
('META', '2022-06-09', '2024-02-15', 424)
('GOOGL', '2007-03-19', '2024-02-15', 4258)
('BRK.B', '1996-05-09', '2024-02-15', 6989)
('AVGO', '2016-02-01', '2024-02-15', 2025)
('LLY', '1982-01-04', '2024-02-15', 9429)
('TSLA', '2011-01-26', '2024-02-15', 3257)
('JPM', '1983-12-30', '2024-02-15', 9559)
('UNH', '1990-03-26', '2024-02-15', 7984)
('V', '2008-03-19', '2024-02-15', 4005)
('XOM', '2001-01-02', '2024-02-15', 5262)
('MA', '2007-03-19', '2024-02-15', 4258)
('JNJ', '1986-01-02', '2024-02-15', 9046)
('PG', '1986-01-02', '2024-02-15', 9047)
('HD', '1984-08-20', '2024-02-15', 9398)
('MRK', '1986-01-02', '2024-02-15', 9046)
('COST', '1986-07-09', '2024-02-15', 8288)
('ABBV', '2013-07-25', '2024-02-15', 2658)
('ADBE', '1986-08-14', '2024-02-15', 8891)
('CRM', '2004-06-23', '2024-02-15', 4259)

# Get price momentum and filter by mkt share
## Get price momentum

In [8]:
df_p_history = pd.read_csv(file_name["price"])
df_p_history["date"] = pd.to_datetime(df_p_history["date"])

## Remove stocks that didn't last until the recent
dt_cutoff = pd.to_datetime(DATE_CUTOFF)
df_p_history["max_date"] = df_p_history.groupby("stock")["date"].transform('max')
df_p_history = (df_p_history[df_p_history["max_date"] >= dt_cutoff])
max_date = df_p_history[df_p_history["max_date"] == df_p_history["date"]]
print("Data of last day: \n")
display(max_date['date'].value_counts())

## Get the starting and end dates to compute the momentum
# The starting date is the DOM_TRAIDING day of MONTH_ACT, and the end date is
# the DOM_TRAIDING day of MONTH_PREV. The two may not be the same calendar DOM
df_p_history = df_p_history.sort_values(["stock", "date"])
df_p_history["year"] = df_p_history.date.dt.year
df_p_history["month"] = df_p_history.date.dt.month
df_p_history["dom_trading"] = df_p_history.groupby(["stock", "year", "month"])[
    "date"
].rank()
df_p_history_prev = df_p_history[
    (df_p_history.month == MONTH_PREV) & (df_p_history.dom_trading == DOM_TRADING)
].copy()
df_p_history_curr = df_p_history[
    (df_p_history.month == MONTH_ACT) & (df_p_history.dom_trading == DOM_TRADING)
].copy()

## Use year_prev to join two datasets.
# For 6month range, if MONTH_ACT is Jul-Dec, the year (year_prev) of MONTH_PREV is
# the same year; otherwise its the previous year
df_p_history_curr["year_prev"] = (
    df_p_history_curr.year if MONTH_ACT - 6 > 0 else df_p_history_curr.year - 1
)

df_p_history_prev["year_prev"] = df_p_history_prev.year
cols = ["date", "close", "stock", "year_prev"]
df_p_history_mmt = pd.merge(
    df_p_history_prev[cols],
    df_p_history_curr[cols + ["year"]],
    on=["stock", "year_prev"],
    suffixes=["_prev", ""],
)

## Get the momentum
df_p_history_mmt["stock_price_mmt_6m"] = (
    df_p_history_mmt["close"] / df_p_history_mmt["close_prev"] - 1
)

Data of last day: 



date
2024-02-15    503
Name: count, dtype: int64

## Get top N by THIS YEAR'S market cap

In [9]:
# Choose only the top N companies of SP500 to start with
df_rank = pd.read_excel('sp500_fulllist_ranked.xlsx', engine='openpyxl', sheet_name=str(YEAR_ACT))
top_stocks = df_rank[df_rank['rank'] <= N_TOP_BY_MKT_CAP].stock.values
df_p_history_mmt = df_p_history_mmt[df_p_history_mmt.stock.isin(top_stocks)]

print(f'{df_p_history_mmt.stock.nunique()} of stocks after filtering by MKT CAP')

299 of stocks after filtering by MKT CAP


In [48]:
print(
    f"The stock with the lowest market cap is {df_rank[df_rank['rank'] == N_TOP_BY_MKT_CAP].stock.values}"
    )

The stock with the lowest market cap is ['LYB']


# Apply the magic formula for a certain year
## Look for anoumalous mmt to detect split

In [10]:
df_row = df_p_history_mmt[df_p_history_mmt.year == YEAR_ACT]

In [12]:
## Check mmt anomalies manually
display(df_row.sort_values('stock_price_mmt_6m').head())
display(df_row.sort_values('stock_price_mmt_6m').tail())

# # Check stocks with super low mmt on tradingview to see if they had a split
# STOCK_SPLIT = ['SRE', 'CPRT' ]  # And their momentum is low; just remove them
# stocks_split = STOCK_SPLIT
# df_row = df_row[~df_row.stock.isin(stocks_split)]

,date_prev,close_prev,stock,year_prev,date,close,year,stock_price_mmt_6m
9092,2023-08-15,140.04,SRE,2023,2024-02-15,70.50,2024,-0.496572
2479,2023-08-15,88.08,CPRT,2023,2024-02-15,49.73,2024,-0.435400
232,2023-08-15,82.43,ADM,2023,2024-02-15,54.31,2024,-0.341138
1985,2023-08-15,428.08,CHTR,2023,2024-02-15,294.14,2024,-0.312885
5065,2023-08-15,495.31,HUM,2023,2024-02-15,363.98,2024,-0.265147


,date_prev,close_prev,stock,year_prev,date,close,year,stock_price_mmt_6m
577,2023-08-15,111.35,AMD,2023,2024-02-15,176.76,2024,0.587427
6602,2023-08-15,301.95,META,2023,2024-02-15,484.03,2024,0.603014
7480,2023-08-15,439.40,NVDA,2023,2024-02-15,726.58,2024,0.653573
7748,2023-08-15,215.66,PANW,2023,2024-02-15,366.88,2024,0.701196
9850,2023-08-15,44.08,UBER,2023,2024-02-15,81.39,2024,0.846416


## Filter by momentum

In [13]:
mmt_var = MMT_VAR
top_by_mmt = TOP_BY_MMT
df_top_by_mmt = (
    df_row.sort_values(mmt_var, ascending=False).iloc[: round(len(df_row) * top_by_mmt), :]
)
print(f'{df_top_by_mmt.stock.nunique()} of stocks after filtering by momentum')

59 of stocks after filtering by momentum


## Download price book ratio of filtered stocks

In [16]:
## A test case
# df_sp500_list_2023 = pd.read_excel('magic_stocks.xlsx', engine='openpyxl', sheet_name='2023')
# stock_list = df_sp500_list_2023.stock.values
# print(stock_list)

stock_list = df_top_by_mmt.stock.unique().tolist()

In [17]:
query = f"""
SELECT act_symbol as stock, `date`, book_value_per_share
FROM `balance_sheet_equity`
WHERE act_symbol IN ('{"', '".join(stock_list)}')
    AND period = 'Quarter'
    AND `date` > '2023-01-01' 
ORDER BY `date` DESC
LIMIT 1000;
"""
# Download data from https://www.dolthub.com/repositories/post-no-preference/earnings/query/master
# And save to file
bv_file_name = f"dolthub_bv_{str(YEAR_ACT)}.csv"

print(query, '\n', bv_file_name)


SELECT act_symbol as stock, `date`, book_value_per_share
FROM `balance_sheet_equity`
WHERE act_symbol IN ('UBER', 'PANW', 'NVDA', 'META', 'AMD', 'ALL', 'PGR', 'AVGO', 'FICO', 'ANET', 'CMG', 'MPWR', 'WTW', 'NFLX', 'CRM', 'QCOM', 'NOW', 'LRCX', 'LLY', 'URI', 'IT', 'KLAC', 'EFX', 'TDG', 'TT', 'TRV', 'AMAT', 'BX', 'INTU', 'GWW', 'IBM', 'AXP', 'IR', 'CDNS', 'GE', 'COST', 'DIS', 'PSX', 'HIG', 'BSX', 'C', 'SNPS', 'PH', 'HLT', 'COF', 'CTAS', 'PCAR', 'INTC', 'ROST', 'ETN', 'MSFT', 'CB', 'SPG', 'WM', 'MU', 'ADSK', 'BK', 'ISRG', 'CEG')
    AND period = 'Quarter'
    AND `date` > '2023-01-01' 
ORDER BY `date` DESC
LIMIT 1000;
 
 dolthub_bv_2024.csv


In [19]:
# Read downloaded .csv from the website above
df_bv = pd.read_csv(bv_file_name)
df_stock_sub = get_quarter(df_bv)
df_stock_sub = df_stock_sub.drop(['DATE', 'dayofyear', ], axis=1).sort_values(['date', 'stock'])

## Check if there is any anomalous pbs
## Replace inf pb to 0
# df_stock_sub = df_stock_sub.replace(np.inf, 0)
# Remove stocks with over MAX_QUARTERS_NEG_PB quarters with neg equity (pb)
# df_stock_sub_neg = df_stock_sub[df_stock_sub.price_to_book_ratio < 0].groupby('stock').size()
# stocks_sub_neg = df_stock_sub_neg[df_stock_sub_neg >= MAX_QUARTERS_NEG_PB].index.values
# df_stock_sub = df_stock_sub[~df_stock_sub.stock.isin(stocks_sub_neg)]
# print(f'{df_stock_sub.stock.nunique()} stocks remained after filtering by neg BV')

## Get data for the quarter needed
df_pb_history = df_stock_sub
df_pb_quarter = (df_pb_history[df_pb_history.index.str.endswith(QUARTER_BV)][['date', 'book_value_per_share', 'stock', 'year']]
                     .reset_index(drop=True)
                     .rename({'year': 'year_prev'}, axis=1)
                )
# Filter out non-positive BV
df_pb_quarter = df_pb_quarter[df_pb_quarter['book_value_per_share'] > 0]

print(f'{df_pb_quarter.stock.nunique()} stocks with the right quarter and positive BV', 
      f'\nStocks without {QUARTER_BV} data or with non-positive BV are {set(df_pb_history.stock.unique()) - set(df_pb_quarter.stock.unique())}'
     )

55 stocks with the right quarter and positive BV 
Stocks without Q3 data or with non-positive BV are {'HLT', 'TDG', 'FICO'}


### Manually check and fill stocks with BV failure
I only filled data for the last quarter for prediction only

In [20]:
# df_pb_quarter

In [21]:
# stocks_failed_pb = ['GM', 'HSY', 'CSGP', 'STT' ]  #  anom['Failed_P']
# stocks_failed_pb_bv = [48.95, 15.26, 16.49, 69.7]   # get from https://www.macrotrends.net/

# df_stocks_failed_pb = pd.DataFrame([
#     [df_pb_quarter.date.max()] * len(stocks_failed_pb),
#     [np.nan] * len(stocks_failed_pb),
#     stocks_failed_pb_bv,
#     stocks_failed_pb,
#     [df_pb_quarter.year_prev.max()] * len(stocks_failed_pb)
# ]).T
# df_stocks_failed_pb.columns = df_pb_quarter.columns

# df_pb_quarter = df_pb_quarter.append(df_stocks_failed_pb)
# print(f'{df_pb_quarter.stock.nunique()} stocks after manully filling in those failing in PB download')

## Merge Data

In [22]:
df_p_pb = pd.merge(df_p_history_mmt[['stock', 'year_prev', 'date', 'close', 
                                     'year', 'stock_price_mmt_6m']], 
                   df_pb_quarter, 
                   on=['stock', 'year_prev'], 
                   suffixes=['', '_pb'])\
            .rename({'close': 'stock_price'}, axis=1)\
            .drop(['year_prev'], axis=1)

df_p_pb['price_to_book_ratio'] = df_p_pb['stock_price'] / df_p_pb['book_value_per_share']

## Pick stocks

In [56]:
stocks_invested = pick_stocks(df_p_pb, 
                              cash_to_invest=TOTAL_CASH,
                              n_stocks=N_STOCKS, 
                              mmt_var=MMT_VAR, 
                             )
stocks_invested['shares'] = stocks_invested['shares'].round()
stocks_invested.sort_values('stock').reset_index(drop=True)

# https://finviz.com/screener.ashx?v=150&f=idx_sp500,ta_perf_26w20o&ft=4&o=pb

,stock,shares,stock_price,price_to_book_ratio,book_value_per_share,stock_price_mmt_6m_pct
0,ALL,31.0,161.89,3.403911,47.56,54.0
1,AMAT,27.0,187.66,10.402439,18.04,34.0
2,AMD,28.0,176.76,5.194240,34.03,59.0
3,ANET,19.0,266.50,12.678402,21.02,48.0
4,AXP,24.0,212.53,5.667467,37.50,31.0
5,BK,90.0,55.29,1.175133,47.05,25.0
6,BSX,76.0,66.09,5.060490,13.06,28.0
7,BX,38.0,130.25,5.299024,24.58,34.0
8,CB,20.0,251.06,1.780820,140.98,26.0
9,CEG,38.0,131.50,3.522636,37.33,24.0


### Sanity check
- Check where a stock is removed below
- If a stock is in finviz filter, but not here, it's possibly due to
  - Filter of Market Cap (only the top N is used)
  - Filter of PB (only the top M is used)
- If a stock is here but not finviz filter, it's possibly due to
  - Filter of momentum (e.g., mmt < 30%>)

finvis filter: https://finviz.com/screener.ashx?v=151&f=idx_sp500,ta_perf_26w30o&ft=4&o=ticker&r=41

In [50]:
stock = 'QCOM'
print(stock in df_sp500_list.stock.values,
      stock in df_p_history.stock.values,
      stock in df_p_history_mmt.stock.values,
      stock in df_row.stock.values,
      stock in df_top_by_mmt.stock.values,
      )
df_mmt = df_row.sort_values(mmt_var, ascending=False).reset_index(drop=True)
print(f'Rank of {stock} by momentum: {df_mmt[df_mmt.stock == stock].index.values[0]}')

True True True True True
Rank of QCOM by momentum: 15


# Backup

## Use API to download fundamental data

In [ ]:
with open('api.my', 'r') as hf:
    fmg_api_key = hf.read()  # financialmodelingprep, no quaterly data
def get_jsonparsed_data(url):
    headers = {
        'Content-Type': 'application/json'
    }
    requestResponse = requests.get(url, headers=headers)
    return requestResponse.json()
# def get_jsonparsed_data(url):
#     response = urlopen(url, cafile=certifi.where())
#     data = response.read().decode("utf-8")
#     return json.loads(data)


# APIs
av_api_key = ''  # alphavantage, Working. Seems not straightforward to cancel subscription
pg_api_key = ''  # Polygon, can't retrieve fundamental data
tg_api_key = ''  # tiingo, fundamental data requires add-on and monthly subs
# Other apis: https://www.fmpcloud.io/plans/, https://data.nasdaq.com/databases/SF1#usage, https://github.com/theOGognf/finagg (promising)
# More at https://www.reddit.com/r/algotrading/comments/1akqgki/is_there_an_inexpensive_api_for_company/, 
# https://www.reddit.com/r/algotrading/comments/122ixxl/cheapest_quote_and_historical_data_provider_for_a/


bv_raw = {}
bv_list = []
stock_list = df_top_by_mmt['stock'].unique()
for symbol in stock_list[:1]:
    # url = (f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?period=annual&apikey={api_key}")
    # url = f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={av_api_key}"
    url = f"https://api.tiingo.com/tiingo/fundamentals/{symbol}/statements?token={tg_api_key}"
    bv_raw[symbol] = get_jsonparsed_data(url)
    for data in bv_raw[symbol]:
        balance_sheet = {i['dataCode']: i['value'] for i in data['statementData']['balanceSheet'] if i['dataCode'] in ['equity', 'sharesBasic']}
        bv_list.append((symbol, data['date'], balance_sheet['equity'], balance_sheet['sharesBasic']))
df_bv = pd.DataFrame(bv_list, columns=('stock', 'date', 'book_value', 'shares_outstanding'))

In [ ]:
# Market Cap data 
us_stock_em = ak.stock_us_spot_em()
us_stock_em[['symbol_prefix', 'symbol']] = us_stock_em['代码'].str.split('.').tolist()
us_stock_em = us_stock_em.rename({'总市值': 'mktCap'}, axis=1)
df_merged = pd.merge(df_bv, us_stock_em[['mktCap', 'symbol']], on='symbol')
df_merged['pb'] = df_merged['mktCap'] / df_merged['book_value']
df_merged

## Backtest

In [12]:
# Remove data with too short length and too late starting date
df_p_pb_size = df_p_pb.groupby('stock').size() 
df_p_pb_cut = df_p_pb[(df_p_pb.year >= YEAR_START) & 
                      (df_p_pb.stock.isin(df_p_pb_size[df_p_pb_size >= MIN_YEARS_TEST].index.values))]
print(f'{df_p_pb_cut.stock.nunique()} of stocks after filtering by length and starting date')

286 of stocks after filtering by length and starting date


In [50]:
top_by_mmt = TOP_BY_MMT
rsl_lt = pd.DataFrame()
# Number of stocks to choose by PB
for n_stocks in [30, 40, 50]:  
    # Invest method:
    # - At the beginning, get the top_by_mmt fraction of stocks ranked by MMT, 
    #   And choose the top n_stocks stocks ranked by PB
    # - After each holding period (PERIOD_HOLD), sell all stocks, and repurchase 
    #   with the initial methods
    cash_to_invest = CASH_FOR_EACH_STOCK * n_stocks
    cash_to_invest_prev = cash_to_invest

    # Run over time
    is_start = True
    period_cnt = 1
    for k, df_row in df_p_pb_cut.groupby('year'):    

        # Initialize
        if is_start:
            stocks_invested = pick_stocks(df_row, 
                              cash_to_invest=cash_to_invest, 
                              n_stocks=n_stocks, 
                              mmt_var=MMT_VAR, 
                              top_by_mmt=top_by_mmt
                             )
            is_start = False
            continue
        
        df_start = pd.merge(df_row[['stock', 'stock_price']], stocks_invested, on='stock', suffixes=('', '_bought'))
        values = (df_start['shares'] * df_start['stock_price']).sum()
        
        # Take action
        cash_to_invest_prev = cash_to_invest
        # Tax
        if values > cash_to_invest_prev:
            cash_to_invest = (values - cash_to_invest_prev) * TAX_FACTOR + cash_to_invest_prev
        else:
            cash_to_invest = values
        stocks_invested = pick_stocks(df_row, 
                                      cash_to_invest=cash_to_invest, 
                                      n_stocks=n_stocks, 
                                      mmt_var=MMT_VAR, 
                                      top_by_mmt=top_by_mmt
                                     )
        rsl_lt = rsl_lt.append(
            pd.DataFrame([k, round(values), 
                          round(values / (CASH_FOR_EACH_STOCK * n_stocks), 3), len(df_start),
                          n_stocks, MMT_VAR, 'lt_1', ], 
                         index=['year', 'value', 'return_overall', 'n_stocks_actual',
                                'n_stocks', 'MMT_VAR', 'method', ]).T) 

        period_cnt += 1
        

In [51]:
# Get SPY history
spy0 = ak.stock_us_daily(symbol="SPY", adjust="").reset_index()
spy = spy0.copy()
spy['year'] = spy.date.dt.year
spy['month'] = spy.date.dt.month
spy['dom_trading'] = spy.groupby(['year', 'month'])['date'].rank()

# Get the first day of MONTH_PREV and of MONTH_ACT, compute the momentum
spy_curr = spy[(spy.month == MONTH_ACT) & (spy.dom_trading == DOM_TRADING)]

spy_curr_cut = spy_curr[spy_curr.year >= YEAR_START]
spy_curr_cut['return_overall'] = (spy_curr_cut['close'] / 
                                  spy_curr_cut[spy_curr_cut.year == YEAR_START]['close'].\
                                  values[0]
                                 ).round(2)

In [52]:
df_compare = pd.merge(rsl_lt, spy_curr_cut[['year', 'return_overall']], 
                      on='year', 
                      suffixes=['', '_spy']).sort_values(['n_stocks', 'year'])
df_compare[df_compare.year.isin(
    [2020, 2021, 2022, 2023]
#     range(2011, 2024)
)][
    ['year', 'n_stocks_actual', 'n_stocks', 'return_overall', 'return_overall_spy']
].sort_values(['year', 'n_stocks'])

,year,n_stocks_actual,N_STOCKS,return_overall,return_overall_spy
24,2020,30,30,2.255,2.48
25,2020,40,40,2.805,2.48
26,2020,50,50,3.042,2.48
27,2021,30,30,3.059,2.88
28,2021,40,40,3.664,2.88
29,2021,50,50,4.032,2.88
30,2022,30,30,5.077,3.46
31,2022,40,40,5.6,3.46
32,2022,50,50,5.846,3.46
33,2023,30,30,5.259,3.14


In [54]:
# The higher the rank means the more years with best return
df_compare['return_overall'] = df_compare['return_overall'].astype(float)
df_compare['rank_n_stock'] = \
    df_compare.groupby('year')['return_overall'].rank()
df_compare.groupby('n_stocks')['rank_n_stock'].mean()

N_STOCKS
30    1.0
40    2.5
50    2.5
Name: rank_n_stock, dtype: float64